In [1]:
import json
import os
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import seaborn as sns
from pylab import rcParams
import random

import matplotlib.pyplot as plt
tf.get_logger().setLevel('ERROR')

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Detect hardware, return appropriate distribution strategy (you can see that it is pretty easy to set up).
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set (always set in Kaggle)
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print('Running on TPU ', tpu.master())
except ValueError:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print('Number of replicas:', strategy.num_replicas_in_sync)

Number of replicas: 1


In [3]:
# Read data from JSON file
with open('/kaggle/input/hackathon-online-nlu-intent-classification2/train.json') as json_file: 
    data_dict = json.load(json_file) 
    
with open('/kaggle/input/hackathon-online-nlu-intent-classification2/valid.json') as json_file: 
    vdata_dict = json.load(json_file)

In [4]:
data_dict

[{'utterance': 'will it be rainy in tenino',
  'slots': 'O O O B-condition_description O B-city',
  'intent': 'GetWeather'},
 {'utterance': 'add steve albini album to my psychedelic rock playlist',
  'slots': 'O B-artist I-artist B-music_item O B-playlist_owner B-playlist I-playlist O',
  'intent': 'AddToPlaylist'},
 {'utterance': 'book a table for 3 at one of the restaurants in washington  d c  in riverview',
  'slots': 'O O O O B-party_size_number O O O O B-restaurant_name I-restaurant_name I-restaurant_name I-restaurant_name I-restaurant_name O B-city',
  'intent': 'BookRestaurant'},
 {'utterance': 'what s the weather forecast for belknap',
  'slots': 'O O O O O O B-city',
  'intent': 'GetWeather'},
 {'utterance': 'play fereydoun farrokhzad best track',
  'slots': 'O B-artist I-artist B-sort B-music_item',
  'intent': 'PlayMusic'},
 {'utterance': 'put kan mikami on pre-party r&b jams',
  'slots': 'O B-artist I-artist O B-playlist I-playlist I-playlist',
  'intent': 'AddToPlaylist'},

In [5]:
train_list = []
val_list = []
# Load data into respective dicts
for data in data_dict:
    train_list.append([data['utterance'],data['intent']])
    
for vdata in vdata_dict:
    val_list.append([vdata['utterance'],vdata['intent']])        

In [6]:
train_list

[['will it be rainy in tenino', 'GetWeather'],
 ['add steve albini album to my psychedelic rock playlist', 'AddToPlaylist'],
 ['book a table for 3 at one of the restaurants in washington  d c  in riverview',
  'BookRestaurant'],
 ['what s the weather forecast for belknap', 'GetWeather'],
 ['play fereydoun farrokhzad best track', 'PlayMusic'],
 ['put kan mikami on pre-party r&b jams', 'AddToPlaylist'],
 ['look for the chu chu and the philly flash picture', 'SearchCreativeWork'],
 ['play the god that failed on vimeo', 'PlayMusic'],
 ['play a soundtrack by mike hindert on spotify', 'PlayMusic'],
 ['want to watch the tv show treeful of starling', 'SearchCreativeWork'],
 ['how much hotter will it be 40 weeks from now in blue island  north dakota',
  'GetWeather'],
 ['put a l lloyd on gaming anthems', 'AddToPlaylist'],
 ['find trailer of jazz impressions of the u s a', 'SearchCreativeWork'],
 ['when is percy jackson playing at the movie theatre',
  'SearchScreeningEvent'],
 ['let me get rese

In [7]:
# Converting to Dataframe for sorting/filtering
train_df = pd.DataFrame(train_list, columns =['text', 'intent'])
val_df = pd.DataFrame(val_list, columns =['text', 'intent'])

In [8]:
train_df.head(10)

,text,intent
0,will it be rainy in tenino,GetWeather
1,add steve albini album to my psychedelic rock ...,AddToPlaylist
2,book a table for 3 at one of the restaurants i...,BookRestaurant
3,what s the weather forecast for belknap,GetWeather
4,play fereydoun farrokhzad best track,PlayMusic
5,put kan mikami on pre-party r&b jams,AddToPlaylist
6,look for the chu chu and the philly flash picture,SearchCreativeWork
7,play the god that failed on vimeo,PlayMusic
8,play a soundtrack by mike hindert on spotify,PlayMusic
9,want to watch the tv show treeful of starling,SearchCreativeWork


In [9]:
val_df.head(10)

,text,intent
0,i d like to see the show onion sportsdome,SearchCreativeWork
1,give the current book 4 stars,RateBook
2,reserve a table for 8 in neighboring hitchland,BookRestaurant
3,show creative video game the boat is full,SearchCreativeWork
4,put once bitten twice bitten into the pulse of...,AddToPlaylist
5,what time is the ladies diplomat playing,SearchScreeningEvent
6,i want to book a bistro that serves pasta sala...,BookRestaurant
7,play a song from 2003,PlayMusic
8,rate current novel two stars,RateBook
9,add the field album to my romantic evening album,AddToPlaylist


In [10]:
trainfeatures=train_df.copy()
trainlabels=trainfeatures.pop('intent')
trainfeatures

,text
0,will it be rainy in tenino
1,add steve albini album to my psychedelic rock ...
2,book a table for 3 at one of the restaurants i...
3,what s the weather forecast for belknap
4,play fereydoun farrokhzad best track
...,...
13079,show me the lights out saga
13080,what is a weather of martinique a month from now
13081,i d like to reserve a spot at a bakery in guya...
13082,please find me the work serious awesomeness


In [11]:
trainlabels.head(10)

0            GetWeather
1         AddToPlaylist
2        BookRestaurant
3            GetWeather
4             PlayMusic
5         AddToPlaylist
6    SearchCreativeWork
7             PlayMusic
8             PlayMusic
9    SearchCreativeWork
Name: intent, dtype: object

In [12]:
trainfeatures=trainfeatures.values

In [13]:
trainfeatures

array([['will it be rainy in tenino'],
       ['add steve albini album to my psychedelic rock playlist'],
       ['book a table for 3 at one of the restaurants in washington  d c  in riverview'],
       ...,
       ['i d like to reserve a spot at a bakery in guyana in feb'],
       ['please find me the work  serious awesomeness'],
       ['play me a song by dj paul elstak in 1988 on zvooq']],
      dtype=object)

In [14]:
from sklearn.preprocessing import LabelBinarizer
binarizer=LabelBinarizer()
trainlabels=binarizer.fit_transform(trainlabels.values)

In [15]:
validfeatures=val_df.copy()
validlabels=validfeatures.pop("intent")

validfeatures=validfeatures.values

validlabels=binarizer.transform(validlabels.values)

In [16]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12'

In [17]:
map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/2',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/2',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/2',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2


In [18]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [19]:
text_test = trainfeatures[0]
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_type_ids', 'input_mask', 'input_word_ids']
Shape      : (1, 128)
Word Ids   : [  101  2097  2009  2022 16373  1999  2702  5740   102     0     0     0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [20]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [21]:
bert_results = bert_model(text_preprocessed)

In [22]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(7, activation=None, name='classifier')(net)
    return tf.keras.Model(text_input, net)

In [23]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(trainfeatures[0]))
print(tf.keras.activations.softmax(bert_raw_result))

tf.Tensor(
[[0.04178241 0.09747405 0.06631422 0.24065971 0.20936538 0.05806822
  0.28633603]], shape=(1, 7), dtype=float32)


In [24]:
classifier_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [25]:
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = tf.metrics.CategoricalAccuracy()

In [26]:
epochs=5
optimizer=tf.keras.optimizers.Adam(1e-5)
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [27]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=trainfeatures,y=trainlabels,
                               validation_data=(validfeatures,validlabels),
                               batch_size=32,
                               epochs=epochs)

Training model with https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Epoch 1/5
409/409 [==============================] - 392s 856ms/step - loss: 0.3079 - categorical_accuracy: 0.9142 - val_loss: 0.0220 - val_categorical_accuracy: 0.9900
Epoch 2/5
409/409 [==============================] - 353s 864ms/step - loss: 0.0401 - categorical_accuracy: 0.9886 - val_loss: 0.0120 - val_categorical_accuracy: 0.9950
Epoch 3/5
409/409 [==============================] - 351s 857ms/step - loss: 0.0225 - categorical_accuracy: 0.9934 - val_loss: 0.0105 - val_categorical_accuracy: 0.9950
Epoch 4/5
409/409 [==============================] - 350s 856ms/step - loss: 0.0145 - categorical_accuracy: 0.9956 - val_loss: 0.0121 - val_categorical_accuracy: 0.9900
Epoch 5/5
409/409 [==============================] - 350s 855ms/step - loss: 0.0098 - categorical_accuracy: 0.9974 - val_loss: 0.0128 - val_categorical_accuracy: 0.9950


In [28]:
scores = classifier_model.evaluate(validfeatures,validlabels, verbose=0)
print("Accuracy: " , (scores[1] * 100))

Accuracy:  99.50000047683716


In [29]:
df_sub = pd.read_csv('/kaggle/input/hackathon-online-nlu-intent-classification2/test.csv')
#df_sub = df_sub.dropna()
#df_sub2 = pd.read_csv('/kaggle/input/hackathon-online-nlu-intent-classification2/small_test.csv')
#df_sub2

In [30]:
#count_nan = df_sub2['utterance'].isna().sum()
#count_nan

In [31]:
X_sub = df_sub[['utterance']].to_numpy().reshape(-1)
X_sub

array(['add this tune to my playlist guest list mashable',
       'i d like to listen to the song the natural farmer',
       'i need a table for 8 people at a restaurant in district of columbia',
       'open the canciones del recuerdo playlist and play songs',
       'i want to hear something from the top-fifty by jose pasillas',
       'i rate the mathematical magpie chronicle a 0 of 6',
       'show the movie schedule of animated movies close by',
       'what time is careful  he might hear you playing at the cinema',
       'i m looking to get a seat at a brasserie in togo',
       'i need a table for 8 during midday in montana',
       'play rain by subliminal on groove shark',
       'what animated movies are playing close by at four o clock',
       'add album to bajo las estrellas',
       'i am giving this novel 3 stars',
       'i need to episodes for the tv series  fast food nation',
       'book a reservation for seven people at a steakhouse in kenya',
       'reserve a sp

In [32]:
binarizer.classes_

array(['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'PlayMusic',
       'RateBook', 'SearchCreativeWork', 'SearchScreeningEvent'],
      dtype='<U20')

In [33]:
test_pre = classifier_model.predict(X_sub)
pred_class = np.argmax(test_pre, axis=1)
pred_class

10/10 [==============================] - 4s 308ms/step


array([0, 5, 1, 3, 3, 4, 6, 6, 1, 1, 3, 6, 0, 4, 5, 1, 1, 6, 0, 6, 3, 5,
       0, 4, 4, 4, 1, 2, 2, 5, 1, 6, 4, 1, 0, 5, 4, 0, 5, 6, 4, 1, 6, 1,
       1, 5, 2, 6, 6, 5, 1, 6, 0, 5, 1, 4, 5, 0, 6, 2, 2, 2, 4, 6, 6, 6,
       6, 2, 5, 0, 0, 3, 6, 2, 0, 1, 5, 4, 2, 0, 4, 2, 3, 1, 5, 6, 0, 4,
       6, 1, 3, 3, 4, 2, 3, 0, 4, 0, 1, 3, 4, 0, 2, 6, 3, 2, 6, 4, 0, 3,
       2, 0, 4, 4, 3, 3, 4, 3, 4, 2, 0, 1, 3, 1, 3, 3, 3, 0, 4, 0, 4, 0,
       4, 0, 1, 2, 5, 2, 0, 5, 4, 5, 0, 3, 0, 1, 5, 4, 3, 4, 2, 5, 0, 1,
       4, 1, 1, 5, 2, 3, 5, 5, 3, 2, 6, 0, 2, 2, 1, 0, 3, 0, 3, 1, 0, 1,
       0, 6, 5, 5, 5, 4, 0, 5, 5, 2, 0, 2, 0, 3, 4, 6, 6, 3, 3, 4, 0, 1,
       1, 5, 3, 6, 6, 1, 0, 3, 5, 1, 2, 1, 3, 6, 5, 6, 2, 4, 6, 2, 2, 2,
       3, 2, 6, 1, 4, 4, 1, 6, 6, 6, 0, 1, 3, 0, 5, 0, 3, 3, 1, 1, 5, 2,
       1, 3, 4, 2, 1, 4, 4, 0, 0, 0, 0, 6, 2, 6, 3, 4, 2, 5, 4, 6, 3, 0,
       0, 5, 3, 6, 1, 5, 4, 2, 0, 0, 2, 2, 2, 6, 1, 5, 3, 3, 5, 1, 5, 3,
       1, 2, 6, 2, 6, 6, 0, 1, 6, 2, 0, 1, 4, 1])

In [34]:
prediction = []
for i in pred_class:
    if i == 0:
        prediction.append("AddToPlaylist")
    elif i == 1:
        prediction.append("BookRestaurant")
    elif i == 2:
        prediction.append("GetWeather")
    elif i == 3:
        prediction.append("PlayMusic")
    elif i == 4:
        prediction.append("RateBook")
    elif i == 5:
        prediction.append("SearchCreativeWork")
    elif i == 6:
        prediction.append("SearchScreeningEvent") 
              
#for i in range(280):
    #test_list = ["AddToPlaylist", "BookRestaurant", "GetWeather", "PlayMusic", "RateBook", "SearchCreativeWork", "SearchScreeningEvent"]  
    #prediction.append(np.nan)

In [35]:
submit = pd.DataFrame({'id':df_sub["id"],
                      'intent':prediction}) 

submit.to_csv('submission.csv', index=False)

In [36]:
submit
#submit.head(20)

,id,intent
0,1,AddToPlaylist
1,2,SearchCreativeWork
2,3,BookRestaurant
3,4,PlayMusic
4,5,PlayMusic
...,...,...
295,296,GetWeather
296,297,AddToPlaylist
297,298,BookRestaurant
298,299,RateBook


In [37]:
model = classifier_model
model.save('modelIntent.h5')